<a href="https://colab.research.google.com/github/Kgan3039/stock-prediction-bert/blob/main/BertModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets scikit-learn torch


In [ ]:
import os
#disable WAndB
os.environ["WANDB_MODE"] = "disabled"
# List all files in the current working directory
print(os.listdir())


['.config', 'Networks (1).py', 'train2D.py', 'DataHandeling.py', 'Inference2D.py', 'create_sequence_metadata.py', 'losses.py', 'Networks.py', 'Params.py', 'utils.py', 'sample_data']


In [ ]:
import torch

print(os.getcwd())
print(torch.cuda.is_available())

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


/content
True


In [ ]:
import pandas as pd

# Reload the train dataset without headers, skipping the first row
train_df = pd.read_csv('stock_data1.csv', header=0)

# Rename columns to reflect their content correctly
train_df.columns = ['text', 'label']

# Drop any row that still contains the column names if it exists
train_df = train_df[train_df['label'] != 'Sentiment']

# Ensure the label column is numeric
train_df['label'] = pd.to_numeric(train_df['label'], errors='coerce')

# Drop rows with NaN labels after conversion
train_df = train_df.dropna(subset=['label']).astype({'label': 'int'})

# Load the validation dataset and rename columns appropriately
val_df = pd.read_csv('twitter_validation.csv', header=None)
val_df = val_df.rename(columns={val_df.columns[2]: 'label', val_df.columns[3]: 'text'})

# Drop unnecessary columns (0 and 1)
val_df = val_df[['text', 'label']]

# Apply label mapping to convert string labels to numeric labels
label_mapping = {
    'Neutral': 0,
    'Positive': 1,
    'Negative': -1,
    'Irrelevant': 2
}
val_df['label'] = val_df['label'].replace(label_mapping)

# Ensure the label column is numeric
val_df['label'] = pd.to_numeric(val_df['label'], errors='coerce')

# Drop rows with NaN labels after conversion
val_df = val_df.dropna(subset=['label']).astype({'label': 'int'})

# Check the size of the filtered dataset
print("Filtered train_df size:", len(train_df))
print("Filtered val_df size:", len(val_df))

# Ensure dataset is not empty before conversion
if len(train_df) == 0 or len(val_df) == 0:
    raise ValueError("Filtered dataset is still empty. Please double-check your dataset and filtering criteria.")

# Remap labels from {-1, 1} to {0, 1}
train_df['label'] = train_df['label'].replace({-1: 0, 1: 1})
val_df['label'] = val_df['label'].replace({-1: 0, 1: 1})

# Filter out any unexpected labels that aren't 0 or 1
train_df = train_df[train_df['label'].isin([0, 1])]
val_df = val_df[val_df['label'].isin([0, 1])]

# Verify the label distribution
print("Train label distribution after remapping:\n", train_df['label'].value_counts())
print("Validation label distribution after remapping:\n", val_df['label'].value_counts())

# Convert to HuggingFace dataset
from datasets import Dataset
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

# Print an example to verify dataset conversion
if len(train_dataset) > 0:
    print("First item in train_dataset:", train_dataset[0])
else:
    raise ValueError("train_dataset is empty after conversion.")

/tmp/ipython-input-1134195521.py:32: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  val_df['label'] = val_df['label'].replace(label_mapping)


Filtered train_df size: 5791
Filtered val_df size: 1000
Train label distribution after remapping:
 label
1    3685
0    2106
Name: count, dtype: int64
Validation label distribution after remapping:
 label
0    551
1    277
Name: count, dtype: int64
First item in train_dataset: {'text': 'Kickers on my watchlist XIDE TIT SOQ PNK CPW BPZ AJ  trade method 1 or method 2, see prev posts', 'label': 1}


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

# Load tokenizer
model_ckpt = "bert-base-uncased"  # Specify model checkpoint
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

# Tokenize the dataset
def tokenize_function(examples):
   return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=128)  # Increase max_length if needed

# Tokenize the training and validation datasets
train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)

# Verify the label set
print("Unique labels in train_dataset:", set(train_dataset['label']))
print("Unique labels in val_dataset:", set(val_dataset['label']))

# Set the number of labels
num_labels = 2

# Load pre-trained BERT model and adjust for sequence classification
try:
    model = AutoModelForSequenceClassification.from_pretrained(model_ckpt, num_labels=num_labels).to(device)
except Exception as e:
    print("Error during model loading:", e)
    raise

#Freeze base model layers
for name, param in model.named_parameters():
    if "layer.10" in name or "layer.11" in name:  # Unfreeze last two layers
        param.requires_grad = True
    elif "classifier" in name:
        param.requires_grad = True
    else:
        param.requires_grad = False



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/5791 [00:00<?, ? examples/s]

Map:   0%|          | 0/828 [00:00<?, ? examples/s]

Unique labels in train_dataset: {0, 1}
Unique labels in val_dataset: {0, 1}


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from sklearn.metrics import precision_score, recall_score, classification_report, accuracy_score, f1_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(-1)
    acc = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='weighted')
    precision = precision_score(labels, predictions, average='weighted')
    recall = recall_score(labels, predictions, average='weighted')
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

In [ ]:
# Ensure labels are numeric
num_labels = len(set(train_dataset['label']))

# Model definition
model = AutoModelForSequenceClassification.from_pretrained(model_ckpt, num_labels=num_labels).to(device)

# Freezing base model parameters to fine-tune classifier only
for param in model.base_model.parameters():
    param.requires_grad = False

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-6,  # Reduce learning rate
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=15,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=100,
    save_total_limit=3,
    load_best_model_at_end=True,
)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


TypeError: TrainingArguments.__init__() got an unexpected keyword argument 'evaluation_strategy'

In [ ]:
from torch.nn import CrossEntropyLoss
from transformers import Trainer
import torch

# Subclassing Trainer to override compute_loss with weighted cross-entropy
class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.get("labels")
        # Forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")

        # Apply weighted cross-entropy loss
        class_weights = torch.tensor([0.5, 1.5]).to(self.args.device)  # Adjust weights accordingly
        loss_fn = CrossEntropyLoss(weight=class_weights)
        loss = loss_fn(logits, labels)

        return (loss, outputs) if return_outputs else loss

# Load model before initializing the Trainer
num_labels = len(set(train_dataset['label']))
model = AutoModelForSequenceClassification.from_pretrained(model_ckpt, num_labels=num_labels)
model = model.to(device)

# Ensure base model layers are frozen (optional)
for param in model.base_model.parameters():
    param.requires_grad = False

# Initialize the custom Trainer with the overridden compute_loss function
trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Print label information and start training
print("Number of labels:", num_labels)
print("Label set in train_dataset:", set(train_dataset['label']))

try:
    trainer.train()
except RuntimeError as e:
    print("RuntimeError during training:", e)
    print("Consider using CUDA_LAUNCH_BLOCKING=1 for better debug messages.")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-9-102fba48c1cd>:30: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `WeightedTrainer.__init__`. Use `processing_class` instead.
  trainer = WeightedTrainer(


Number of labels: 2
Label set in train_dataset: {0, 1}


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.572700,0.710070,0.334541,0.167725,0.111918,0.334541
2,0.471000,0.823488,0.334541,0.167725,0.111918,0.334541
3,0.465700,0.858356,0.334541,0.167725,0.111918,0.334541
4,0.449500,0.874630,0.334541,0.167725,0.111918,0.334541
5,0.453100,0.874973,0.334541,0.167725,0.111918,0.334541
6,0.463800,0.871487,0.334541,0.167725,0.111918,0.334541
7,0.463000,0.866812,0.334541,0.167725,0.111918,0.334541
8,0.462400,0.866667,0.334541,0.167725,0.111918,0.334541
9,0.444000,0.866434,0.334541,0.167725,0.111918,0.334541
10,0.452800,0.866282,0.334541,0.167725,0.111918,0.334541


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/m

In [ ]:
for name, param in model.named_parameters():
  print(f"Layer: {name} | Requires Grad: {param.requires_grad}")

Layer: bert.embeddings.word_embeddings.weight | Requires Grad: False
Layer: bert.embeddings.position_embeddings.weight | Requires Grad: False
Layer: bert.embeddings.token_type_embeddings.weight | Requires Grad: False
Layer: bert.embeddings.LayerNorm.weight | Requires Grad: False
Layer: bert.embeddings.LayerNorm.bias | Requires Grad: False
Layer: bert.encoder.layer.0.attention.self.query.weight | Requires Grad: False
Layer: bert.encoder.layer.0.attention.self.query.bias | Requires Grad: False
Layer: bert.encoder.layer.0.attention.self.key.weight | Requires Grad: False
Layer: bert.encoder.layer.0.attention.self.key.bias | Requires Grad: False
Layer: bert.encoder.layer.0.attention.self.value.weight | Requires Grad: False
Layer: bert.encoder.layer.0.attention.self.value.bias | Requires Grad: False
Layer: bert.encoder.layer.0.attention.output.dense.weight | Requires Grad: False
Layer: bert.encoder.layer.0.attention.output.dense.bias | Requires Grad: False
Layer: bert.encoder.layer.0.attenti

In [ ]:
trainer.evaluate()


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Error: You must call wandb.init() before wandb.log()

In [ ]:
model.save_pretrained('./finetuned-bert-model')
tokenizer.save_pretrained('./finetuned-bert-tokenizer')


('./finetuned-bert-tokenizer/tokenizer_config.json',
 './finetuned-bert-tokenizer/special_tokens_map.json',
 './finetuned-bert-tokenizer/vocab.txt',
 './finetuned-bert-tokenizer/added_tokens.json',
 './finetuned-bert-tokenizer/tokenizer.json')

In [ ]:
sentiment_predictions = []

In [ ]:
import yfinance as yf
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [ ]:
ticker = "TSLA"
start_date = "2023-01-01"
end_date = "2023-05-01"
stock_data = yf.download(ticker, start=start_date, end=end_date)
stock_data = stock_data[['Adj Close']].reset_index()
stock_data.rename(columns={'Adj Close': 'adj_close'}, inplace=True)


[*********************100%***********************]  1 of 1 completed


In [ ]:
def calculate_weighted_sentiment(df):
    df['normalized_retweets'] = df['retweets'] / df['retweets'].max()
    df['weighted_sentiment'] = df['normalized_retweets'] * df['sentiment']
    daily_sentiment = df.groupby('date')['weighted_sentiment'].sum().reset_index()
    return daily_sentiment

In [ ]:
# Assuming `sentiment_predictions` is a list of dictionaries, you need to convert it to a DataFrame
# Example:
# sentiment_predictions = [{'date': '2023-01-01', 'retweets': 10, 'sentiment': 1}, ...]

# Convert to DataFrame if it's not already one
if isinstance(sentiment_predictions, list):
    sentiment_predictions = pd.DataFrame(sentiment_predictions)

# Check if `sentiment_predictions` is now a DataFrame
print(sentiment_predictions.head())

# Function to calculate weighted sentiment
def calculate_weighted_sentiment(df):
    if not isinstance(df, pd.DataFrame):
        raise ValueError("Input data must be a pandas DataFrame")

    # Ensure required columns are present
    if 'retweets' not in df.columns or 'sentiment' not in df.columns or 'date' not in df.columns:
        raise ValueError("Data must include 'date', 'retweets', and 'sentiment' columns.")

    # Normalize retweet counts
    df['normalized_retweets'] = df['retweets'] / df['retweets'].max()

    # Weighted sentiment score = normalized retweets * sentiment
    df['weighted_sentiment'] = df['normalized_retweets'] * df['sentiment']

    # Aggregate by date to get the daily weighted sentiment
    daily_sentiment = df.groupby('date')['weighted_sentiment'].sum().reset_index()
    return daily_sentiment

daily_sentiment = calculate_weighted_sentiment(sentiment_predictions)

merged_data = pd.merge(stock_data, daily_sentiment, left_on='Date', right_on='date', how='inner')
merged_data = merged_data[['Date', 'adj_close', 'weighted_sentiment']]

Empty DataFrame
Columns: []
Index: []


ValueError: Data must include 'date', 'retweets', and 'sentiment' columns.

In [ ]:
lookback_window = 5
for i in range(1, lookback_window + 1):
    merged_data[f'adj_close_lag_{i}'] = merged_data['adj_close'].shift(i)
    merged_data[f'weighted_sentiment_lag_{i}'] = merged_data['weighted_sentiment'].shift(i)
merged_data = merged_data.dropna()

# Define the target: 1 if price increased, 0 if decreased
merged_data['price_direction'] = (merged_data['adj_close'].diff().shift(-1) > 0).astype(int)
features = [col for col in merged_data.columns if 'lag' in col]
X = merged_data[features]
y = merged_data['price_direction']

NameError: name 'merged_data' is not defined

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
print(f"Stock Prediction Accuracy: {accuracy:.2f}, F1 Score: {f1:.2f}")